In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
from keras import losses

custom_objects = {
    'mse': losses.mean_squared_error
}

model = load_model(
    r'C:/Users/sahil/Desktop/Crop_yeild/blstm_100_future/BLSTM_7_crop_yield_prediction_20.h5',
    custom_objects=custom_objects
)

In [11]:
%pip install pandas

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.6 MB 4.6 MB/s eta 0:00:03
   ------- -------------------------------- 2.1/11.6 MB 4.9 MB/s eta 0:00:02
   ------------ --------------------------- 3.7/11.6 MB 5.9 MB/s eta 0:00:02
   ------------------ --------------------- 5.2/11.6 MB 6.4 MB/s eta 0:00:01
   --------------------- ------------------ 6.3/11.6 MB 6.2 MB/s eta 0:00:01
   --------------------------- ------------ 7.9/11.6 MB 6.3 MB/s eta 0:00:01
   ------------------------------- -------- 9.2/11.6 MB 6.4 MB/s eta 0:00:01
   ------------------------------------ --- 10.5/11.6 MB 6.2 MB/s eta 0:00:01
   ---------------------------------------  11.5/11.6 MB 6.1 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 6.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.1 MB 10.1 MB/s eta 0:00:01
   ------------ --------------------------- 3.4/11.1 MB 8.1 MB/s eta 0:00:01
   ---------------- ----------------------- 4.5/11.1 MB 7.5 MB/s eta 0:00:01
   --------------------- ------------------ 6.0/11.1 MB 7.1 MB/s eta 0:00:01
   ------------------------- -------------- 7.1/11.1 MB 6.9 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.1 MB 6.8 MB/s eta 0:00:01
   ----------------------------------- ---- 10.0/11.1 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/41.2 MB ? eta -:--:--
   - -------------------------------------- 1.6/41.2 MB 7.7 MB/s eta 0:00:06
   -- ------------------------------------- 2.9/41.2 MB 7.0 MB/s eta 0:00:06
   ---- ----------------------------------- 4.2/41.2 MB 6.8 MB/s eta 0:00:06
   ----- -

In [3]:
import pandas as pd

file2 = 'C:/Users/sahil/Desktop/Crop_yeild/maize_country_data_20/maize_India.csv'


df2 = pd.read_csv(file2)



# Backup the original 'crop', 'State', and 'Country' columns
original_columns = df2[['crop', 'State', 'Country']].copy()
# Check for duplicates and remove them if needed
df2 = df2.drop_duplicates()



df2.drop(['crop', 'State', 'Country'], axis=1, inplace=True)
#remove the columns with missing data
df2.dropna(inplace=True)
#remove the variable with VIF greater than a threshold from the data set
#df.drop(['Tmin','Tmax', 'Humidity_min',  'Humidity_max','Solar R_Avg'],axis=1 ,inplace=True)

In [4]:
import random

def data_split(df, train=True):
    sequential_data = []
    season = []
    for index, i in enumerate(df.values):
        season.append(i[:-1].tolist())  # All columns except the last one (yield)
        if len(season) == 240:
            sequential_data.append([np.array(season), i[-1]])  # Last column is the target (yield)
            season.clear()

    if train:
        random.shuffle(sequential_data)

    X = []
    Y = []
    for seq, target in sequential_data:
        X.append(seq)
        Y.append(target)
    return np.array(X), np.array(Y)

In [5]:
# First split the data for supervised learning using the data_split function
X, Y = data_split(df2)
indices = np.arange(len(X))

In [6]:
import joblib
# Load the pre-fitted scaler
scaler = joblib.load('minmax_scaler.save') 

In [9]:


# Concatenate X and Y to preprocess test set
X_reshape = np.reshape(X, (X.shape[0] * X.shape[1], X.shape[2]))
Y_reshape = np.concatenate((Y.reshape(-1, 1), np.zeros((X.shape[0] * X.shape[1] - len(Y), 1))), axis=0)
data_test = np.concatenate((X_reshape, Y_reshape), axis=1)

# Normalize the test set using the training MinMaxScaler
data_test_scaled = scaler.transform(data_test)
X_scaled = data_test_scaled[:, :-1]
X_scaled = np.reshape(X_scaled, (X.shape[0], X.shape[1], X.shape[2]))

# Predict on the test set
yhat = model.predict(X_scaled)

# Denormalize the predictions and actual values
# Append predictions to test input for inverse transformation
yhat_reshape = np.concatenate((yhat, np.zeros((X.shape[0]*X.shape[1] - len(yhat), yhat.shape[1]))), axis=0)
data_pred = np.concatenate((X_reshape, yhat_reshape), axis=1)

# Inverse transform both true and predicted data
data_actual = scaler.inverse_transform(data_test_scaled)
data_pred = scaler.inverse_transform(data_pred)

inv_y = data_actual[:len(Y), -1]
inv_yhat = data_pred[:len(Y), -1]

349/349 ━━━━━━━━━━━━━━━━━━━━ 42s 120ms/step


In [10]:
yhat

array([[0.04234587],
       [0.15919495],
       [0.13366672],
       ...,
       [0.07410336],
       [0.07415499],
       [0.08805815]], dtype=float32)

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

# Calculate R², MSE, MAE, RMSE
r2 = r2_score(inv_y, inv_yhat)
mse = mean_squared_error(inv_y, inv_yhat)
mae = mean_absolute_error(inv_y, inv_yhat)
rmse = sqrt(mse)

# Print results
print(f'R² Score       = {r2:.4f}')
print(f'Mean Squared Error (MSE) = {mse:.4f}')
print(f'Mean Absolute Error (MAE) = {mae:.4f}')
print(f'Root Mean Squared Error (RMSE) = {rmse:.4f}')

R² Score       = -2.4609
Mean Squared Error (MSE) = 122.2460
Mean Absolute Error (MAE) = 9.3737
Root Mean Squared Error (RMSE) = 11.0565


In [12]:
inv_y

array([13.10891 , 18.98763 ,  7.149752, ...,  9.964984, 18.32651 ,
       14.62276 ])

In [13]:
# Create a DataFrame for actual vs predicted values
datapredicted = pd.DataFrame({
    'Actual': inv_y,
    'Predicted': inv_yhat
})

# Print and save to CSV
print(datapredicted.head()) 

      Actual  Predicted
0  13.108910   0.306203
1  18.987630   1.151139
2   7.149752   0.966544
3  13.787640   4.452828
4   3.965434   2.900227


In [16]:
# Flatten the predicted values to match the original data's shape
predicted_flattened = np.repeat(inv_yhat, 240)  # Repeat each prediction for 240 days

# Ensure the length matches the original dataframe
if len(predicted_flattened) > len(df2):
    predicted_flattened = predicted_flattened[:len(df2)]
else:
    predicted_flattened = np.pad(predicted_flattened, (0, len(df2) - len(predicted_flattened)), 'constant', constant_values=np.nan)

# Add predicted values to the dataframe
df2['Predicted_Yield'] = predicted_flattened

# Set Predicted_Yield to 0 where day is not equal    to 240
df2['Predicted_Yield'] = np.where(df2['Day'] != 239, 0, df2['Predicted_Yield'])

# Add original columns back to df2 with predicted_yield included
df2_with_predictions = pd.concat([df2, original_columns], axis=1)
# Save the updated dataframe with predictions
df2_with_predictions.to_csv("updated_with_predictions.csv", index=False)

print("Updated dataset with predictions saved as 'updated_with_predictions.csv'")


Updated dataset with predictions saved as 'updated_with_predictions.csv'
